In [2]:
import datetime
import time
import tarfile

import boto3
import pandas as pd
import numpy as np
from sagemaker import get_execution_role
import sagemaker
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_boston


sm_boto3 = boto3.client("sagemaker")

sess = sagemaker.Session()

region = sess.boto_session.region_name

bucket = sess.default_bucket()  # this could also be a hard-coded bucket name

print("Using bucket " + bucket)



Using bucket sagemaker-us-east-1-769654117898


In [3]:
# we use the Boston housing dataset
data = load_boston()

In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    data.data, data.target, test_size=0.25, random_state=42
)

trainX = pd.DataFrame(X_train, columns=data.feature_names)
trainX["target"] = y_train

testX = pd.DataFrame(X_test, columns=data.feature_names)
testX["target"] = y_test



In [5]:
trainX.head()

trainX.to_csv("boston_train.csv")
testX.to_csv("boston_test.csv")

# send data to S3. SageMaker will take training data from s3
trainpath = sess.upload_data(
    path="boston_train.csv", bucket=bucket, key_prefix="sagemaker/sklearncontainer"
)

testpath = sess.upload_data(
    path="boston_test.csv", bucket=bucket, key_prefix="sagemaker/sklearncontainer"
)



In [6]:
%%writefile script.py

import argparse
import joblib
import os

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor


# inference functions ---------------
def model_fn(model_dir):
    clf = joblib.load(os.path.join(model_dir, "model.joblib"))
    return clf


if __name__ == "__main__":

    print("extracting arguments")
    parser = argparse.ArgumentParser()

    # hyperparameters sent by the client are passed as command-line arguments to the script.
    # to simplify the demo we don't use all sklearn RandomForest hyperparameters
    parser.add_argument("--n-estimators", type=int, default=10)
    parser.add_argument("--min-samples-leaf", type=int, default=3)

    # Data, model, and output directories
    parser.add_argument("--model-dir", type=str, default=os.environ.get("SM_MODEL_DIR"))
    parser.add_argument("--train", type=str, default=os.environ.get("SM_CHANNEL_TRAIN"))
    parser.add_argument("--test", type=str, default=os.environ.get("SM_CHANNEL_TEST"))
    parser.add_argument("--train-file", type=str, default="boston_train.csv")
    parser.add_argument("--test-file", type=str, default="boston_test.csv")
    parser.add_argument(
        "--features", type=str
    )  # in this script we ask user to explicitly name features
    parser.add_argument(
        "--target", type=str
    )  # in this script we ask user to explicitly name the target

    args, _ = parser.parse_known_args()

    print("reading data")
    train_df = pd.read_csv(os.path.join(args.train, args.train_file))
    test_df = pd.read_csv(os.path.join(args.test, args.test_file))

    print("building training and testing datasets")
    X_train = train_df[args.features.split()]
    X_test = test_df[args.features.split()]
    y_train = train_df[args.target]
    y_test = test_df[args.target]

    # train
    print("training model")
    model = RandomForestRegressor(
        n_estimators=args.n_estimators, min_samples_leaf=args.min_samples_leaf, n_jobs=-1
    )

    model.fit(X_train, y_train)

    # print abs error
    print("validating model")
    abs_err = np.abs(model.predict(X_test) - y_test)

    # print couple perf metrics
    for q in [10, 50, 90]:
        print("AE-at-" + str(q) + "th-percentile: " + str(np.percentile(a=abs_err, q=q)))

    # persist model
    path = os.path.join(args.model_dir, "model.joblib")
    joblib.dump(model, path)
    print("model persisted at " + path)
    print(args.min_samples_leaf)



Writing script.py


In [7]:
! python script.py --n-estimators 100 \
                   --min-samples-leaf 2 \
                   --model-dir ./ \
                   --train ./ \
                   --test ./ \
                   --features 'CRIM ZN INDUS CHAS NOX RM AGE DIS RAD TAX PTRATIO B LSTAT' \
                   --target target

extracting arguments
reading data
building training and testing datasets
training model
validating model
AE-at-10th-percentile: 0.26509238095237964
AE-at-50th-percentile: 1.6450047619047652
AE-at-90th-percentile: 4.640090714285712
model persisted at ./model.joblib
2


In [8]:
# We use the Estimator from the SageMaker Python SDK
from sagemaker.sklearn.estimator import SKLearn

FRAMEWORK_VERSION = "0.23-1"

sklearn_estimator = SKLearn(
    entry_point="script.py",
    role=get_execution_role(),
    instance_count=1,
    instance_type="ml.m4.xlarge",#"ml.c5.xlarge",#"ml.m4.xlarge",
    framework_version=FRAMEWORK_VERSION,
    base_job_name="rf-scikit",
    metric_definitions=[{"Name": "median-AE", "Regex": "AE-at-50th-percentile: ([0-9.]+).*$"}],
    hyperparameters={
        "n-estimators": 100,
        "min-samples-leaf": 3,
        "features": "CRIM ZN INDUS CHAS NOX RM AGE DIS RAD TAX PTRATIO B LSTAT",
        "target": "target",
    },
)

import time
tic = time.clock()

# launch training job, with asynchronous call
sklearn_estimator.fit({"train": trainpath, "test": testpath}, wait=False)

toc = time.clock()
print(toc - tic)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:23: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


0.09467700000000012


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:28: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


In [9]:
sklearn_estimator.latest_training_job.wait(logs="None")
artifact = sm_boto3.describe_training_job(
    TrainingJobName=sklearn_estimator.latest_training_job.name
)["ModelArtifacts"]["S3ModelArtifacts"]

print("Model artifact persisted at " + artifact)

from sagemaker.sklearn.model import SKLearnModel

model = SKLearnModel(
    model_data=artifact,
    role=get_execution_role(),
    entry_point="script.py",
    framework_version=FRAMEWORK_VERSION,
)

predictor = model.deploy(instance_type="ml.c5.large", initial_instance_count=1)




2021-12-30 19:44:07 Starting - Launching requested ML instances..........
2021-12-30 19:45:21 Starting - Preparing the instances for training.....................
2021-12-30 19:47:13 Downloading - Downloading input data......
2021-12-30 19:47:47 Training - Downloading the training image....
2021-12-30 19:48:14 Training - Training image download completed. Training in progress..
2021-12-30 19:48:19 Uploading - Uploading generated training model
2021-12-30 19:48:28 Completed - Training job completed
Model artifact persisted at s3://sagemaker-us-east-1-769654117898/rf-scikit-2021-12-30-19-44-03-712/output/model.tar.gz
-----!

In [10]:

n=0
while n<400:
    print(predictor.predict(testX[data.feature_names]))
    n=n+1

[22.6503812  31.94007381 17.83357579 23.61797698 16.81124423 21.37129668
 19.46213752 15.94291389 21.24104246 21.05062513 20.59566111 19.67550238
  8.39862009 21.85981667 19.34187987 25.05954167 18.99944881  8.89008636
 44.68355775 15.27263716 23.91849265 23.67633668 15.01245238 23.52560256
 15.12859015 15.49855794 21.84817442 13.73335155 19.27734672 21.02351456
 20.48238413 23.18741003 29.12035343 20.45508088 14.77991743 15.9726873
 34.53880697 19.37766638 21.30764095 23.90469444 19.78590242 28.35096418
 44.95183275 19.42216771 22.40314921 13.86123424 15.51815119 23.81158896
 18.20821778 29.48817059 21.43161984 33.7091693  16.59605159 26.01817923
 45.69001124 21.77233521 15.81332341 32.34799004 21.52080577 20.65990909
 25.12002222 33.78015895 31.3543406  18.41873719 27.59481465 16.9512127
 13.12180447 23.17948836 29.16539672 14.92331429 20.75199545 27.22520557
 10.04196981 21.89419354 22.25977684  7.74780819 19.85389761 45.54577434
 10.7800496  14.03422089 21.85704755 11.00907367 20.2

In [11]:
sm_boto3.delete_endpoint(EndpointName=predictor.endpoint_name)

{'ResponseMetadata': {'RequestId': 'cbeb10a3-14bf-4074-933e-843a74700175',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'cbeb10a3-14bf-4074-933e-843a74700175',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Thu, 30 Dec 2021 19:52:28 GMT'},
  'RetryAttempts': 0}}